# Week13 Client Project: Text Generation LSTM (Char-level)

**Full training (15 epochs).**

This client-project notebook is modular: replace `data_path` with your text corpus (e.g., a large txt file).

In [ ]:

# Client Project: Char-level LSTM Text Generator
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np, os, random, io, sys
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Path to a text file (replace with larger corpus for production)
data_path = "/mnt/data/sample_corpus.txt"

# If the file doesn't exist, create a placeholder sample file (users should replace this)
if not os.path.exists(data_path):
    sample = (
        "Call me Ishmael. Some years ago—never mind how long precisely—having little or no money in my purse, "
        "and nothing particular to interest me on shore, I thought I would sail about a little and see the watery part of the world."
    )
    with open(data_path, "w") as f:
        f.write(sample)

with open(data_path, "r", encoding="utf-8") as f:
    text = f.read()
print("Corpus length:", len(text))

# Build character indices
chars = sorted(list(set(text)))
char2idx = {c:i for i,c in enumerate(chars)}
idx2char = {i:c for c,i in char2idx.items()}

# Parameters
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
print("Number of sequences:", len(sentences))

# Vectorize
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.float32)
y = np.zeros((len(sentences), len(chars)), dtype=np.float32)
for i, sentence in enumerate(sentences):
    for t, ch in enumerate(sentence):
        X[i, t, char2idx[ch]] = 1.0
    y[i, char2idx[next_chars[i]]] = 1.0

# Model
model = models.Sequential([
    layers.LSTM(256, input_shape=(maxlen, len(chars))),
    layers.Dense(len(chars), activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Callbacks
out_model = os.path.join("/mnt/data", "week13_client_textgen_model.h5")
callbacks = [
    ModelCheckpoint(out_model, save_best_only=True, monitor='loss'),
    EarlyStopping(patience=5, monitor='loss', restore_best_weights=True)
]

# Train
model.fit(X, y, batch_size=128, epochs=15, callbacks=callbacks, verbose=2)

# Save final model (already saved by checkpoint)
print("Model saved to", out_model)

# Generate sample text
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

start_index = random.randint(0, max(1, len(text) - maxlen - 1))
seed = text[start_index: start_index + maxlen]
print("Seed:", seed)
generated = seed
for temp in [0.5, 1.0, 1.2]:
    print("---- temp:", temp)
    gen = seed
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, ch in enumerate(gen[-maxlen:]):
            x_pred[0, t, char2idx[ch]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temp)
        next_char = idx2char[next_index]
        gen += next_char
    print(gen[:800])
